In [10]:
import json
import pandas as pd

from visualize import *

my_dict = {'t1': 'T1','t2': 'T2','t3': 'T3','t4': 'T4','t5': 'T5',
           'v1': 'V1','v2': 'V2','v3': 'V3','v4': 'V4','v5': 'V5',
           'Zh':'Zh', 'Ko':'Ko', 'Es':'Es', 'Fr':'Fr', 'De':'De',
           'It':'It', 'Ar':'Ar', 'Ru':'Ru', 'Ja':'Ja', 'En':'En',
           'n':'Arabic Numeral', 'al':'Lowercase Latin', 'au':'Uppercase Latin',
           'rl':'Lowercase Roman', 'ru':'Uppercase Roman',
           'f':'Ascending', 'r':'Descending',
           'a person of routine and familiarity': 'Routine',
           'a more spontaneous and less reliable person': 'Spontaneous',
           'a person with reserved and lower energy levels': 'Reserved',
           "a competitive person, sometimes skeptical of others' intentions": 'Competitive',
           'a person with emotional stability and consistent moods': 'Stability',
           'an adventurous and creative person': 'Adventurous',
           'an organized person, mindful of details': 'Organized',
           'a person full of energy and positive emotions': 'Energy',
           'a compassionate and cooperative person': 'Compassionate',
           'a person with emotional instability and diverse negative feelings': 'Instability',
           }

my_colors = ['#e6194B', '#42d4f4', '#ffe119', '#3cb44b', '#f032e6', '#fabed4', '#469990', '#dcbeff',
             '#9A6324', '#fffac8', '#800000', '#aaffc3', '#000075', '#a9a9a9', '#000000']

default_color = '#D9DDDC'


In [11]:
reference = extract_reference('reference/full.json')
vis = Visualize('BFI', reference)
data, info = extract_data('save/save.json')


('template', 'language', 'version', 'label', 'order')

In [ ]:
# Plot Prompt Sensitivity Results
for aspect in info:
    for index, value in enumerate(data[aspect].unique()):
        vis.add(data[data[aspect] == value], my_colors[index], my_dict[value])
    vis.plot(f'bfi-{aspect}', True)
    vis.clean()
    

In [ ]:
# Plot Outliers
data_outliers = vis.detect_outlier(data, 0.302, 20)
vis.add(data_outliers[data["Label"] == -1], my_colors[0], 'Outliers')
vis.add(data_outliers[data["Label"] != -1], my_colors[1], 'Inliers')
vis.plot(f'outliers')
vis.clean()


In [ ]:
# Plot Environment Appending Tests
env_data, _ = extract_data('save/environment.json')

with open('dataset/environment.json') as f:
    env_conf = json.load(f)

for key in env_conf:
    vis.add(data, default_color, 'Default')
    for index, emotion in enumerate(env_conf[key].keys()):
        vis.add(env_data[env_data["append_label"] == emotion], my_colors[index], emotion.capitalize())
    vis.plot(f"env-{key}", random_zorder=True, exclude=[0])
    vis.clean()
    

In [ ]:
# Plot Character Appending Tests
chara_data, _ = extract_data('save/character.json')
chara_cot_data, _ = extract_data('save/character.json')
chara_data = pd.concat([chara_data, chara_cot_data])

with open('dataset/character.json') as f:
    chara_conf = json.load(f)

for key in chara_conf:
    vis.add(data, default_color, 'Default')
    for index, chara in enumerate(chara_conf[key].keys()):
        vis.add(chara_data[chara_data["append_label"] == chara], my_colors[index], chara)
    vis.plot(f"chara-{key}", random_zorder=True, exclude=[0])
    vis.clean()
    

In [ ]:
# Plot Personality Tests
personality_list = []
for file in ['personality_portray_cot', 'personality_portray', 'personality_biography_cot',
             'personality_biography', 'personality_qa_cot', 'personality_qa']:
    personality_list.append(extract_data(f'save/{file}.json')[0])
pers_data = pd.concat(personality_list)

with open('dataset/personality.json') as f:
    pers_conf = json.load(f)

for key in pers_conf:
    vis.add(data, default_color, 'Default')
    for index, pers in enumerate(pers_conf[key].keys()):
        vis.add(pers_data[pers_data["append_label"] == pers], my_colors[index], my_dict[pers].capitalize())
    vis.plot(f"pers-{key}", random_zorder=True, exclude=[0])
    vis.clean()
   
 
# Plot Personalities Comparisons
for min_label, max_label in zip(list(pers_conf["minimum"].keys()), list(pers_conf["maximum"].keys())):
    vis.add(data, default_color)
    vis.add(pers_data[pers_data["append_label"] == min_label], my_colors[0], my_dict[min_label].capitalize())
    vis.add(pers_data[pers_data["append_label"] == max_label], my_colors[1], my_dict[max_label].capitalize())
    vis.plot()
    vis.clean()
    

In [ ]:
# Plot COT Results
for file in ['personality_portray', 'personality_biography', 'personality_qa', 'character']:
    cot, _ = extract_data(f'save/{file}_cot.json')
    no_cot, _ = extract_data(f'save/{file}.json')
    vis.add(data, default_color, 'Default')
    vis.add(cot, my_colors[0], 'With COT')
    vis.add(no_cot, my_colors[1], 'Without COT')
    vis.plot(f"cot-{file}", random_zorder=True, exclude=[0])
    vis.clean()
    

In [ ]:
# Statistical Analysis
filename = 'comp.md'
questionnaire = get_questionnaire('BFI')
categories = list(questionnaire['categories'].keys())

# data, info = extract_data('save/save.json')

analysis_data = data

# Remove outliers
analysis_data = vis.detect_outlier(data, 0.302, 20)[data["Label"] != -1]
print(analysis_data.shape)

with open(filename, 'w') as output_file:
    for aspect in info:
        write_df = pd.DataFrame(columns=categories)
        
        for index, value in enumerate(data[aspect].unique()):
            records = list()
            x = data[data[aspect] == value]
            y = data[data[aspect] != value]
            
            for cat in categories:
                sym, diff, _, p_val = hypothesis_testing(x[cat].tolist(), y[cat].tolist())
                
                "==================== Output Template ===================="
                records.append(f'${sym} {diff:.2f}_{{{p_val:.2f}}}$')
                "======================================================="
            
            write_df.loc[my_dict[value]] = records

        output_file.write(f'### {aspect.capitalize()}\n')
        "==================== Output Format ===================="
        output_file.write(write_df.to_markdown())
        # output_file.write(write_df.to_latex(escape=False))
        "======================================================="
        output_file.write('\n\n')
